In [1]:
#Installing Whisper
! pip install git+https://github.com/openai/whisper.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-_vtafu1d
  Running command git clone -q https://github.com/openai/whisper.git /tmp/pip-req-build-_vtafu1d
     |████████████████████████████████| 5.3 MB 27.9 MB/s 
     |████████████████████████████████| 163 kB 59.3 MB/s 
     |████████████████████████████████| 7.6 MB 52.7 MB/s 
  Created wheel for whisper: filename=whisper-1.0-py3-none-any.whl size=1175227 sha256=4f33a0cd674a75c76b027ae59bf7519bf6183e680afabbccb0eeb5e0a0339c96
  Stored in directory: /tmp/pip-ephem-wheel-cache-bmw_uv93/wheels/16/15/89/1c7bb31bd0006793a95549d04785121a8a36daad9158e1e43a
Successfully built whisper


In [2]:
#importing required libraries
import io
import os
import numpy as np

try:
    import tensorflow  
except ImportError:
    pass

import torch
import pandas as pd
import urllib
import tarfile
import whisper
import torchaudio

from scipy.io import wavfile
from tqdm.notebook import tqdm


pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 1000
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
#selecting language of the data Fleurs dataset before download
import ipywidgets as widgets

languages = {"af_za": "Afrikaans", "am_et": "Amharic", "ar_eg": "Arabic", "as_in": "Assamese", "az_az": "Azerbaijani", "be_by": "Belarusian", "bg_bg": "Bulgarian", "bn_in": "Bengali", "bs_ba": "Bosnian", "ca_es": "Catalan", "cmn_hans_cn": "Chinese", "cs_cz": "Czech", "cy_gb": "Welsh", "da_dk": "Danish", "de_de": "German", "el_gr": "Greek", "en_us": "English", "es_419": "Spanish", "et_ee": "Estonian", "fa_ir": "Persian", "fi_fi": "Finnish", "fil_ph": "Tagalog", "fr_fr": "French", "gl_es": "Galician", "gu_in": "Gujarati", "ha_ng": "Hausa", "he_il": "Hebrew", "hi_in": "Hindi", "hr_hr": "Croatian", "hu_hu": "Hungarian", "hy_am": "Armenian", "id_id": "Indonesian", "is_is": "Icelandic", "it_it": "Italian", "ja_jp": "Japanese", "jv_id": "Javanese", "ka_ge": "Georgian", "kk_kz": "Kazakh", "km_kh": "Khmer", "kn_in": "Kannada", "ko_kr": "Korean", "lb_lu": "Luxembourgish", "ln_cd": "Lingala", "lo_la": "Lao", "lt_lt": "Lithuanian", "lv_lv": "Latvian", "mi_nz": "Maori", "mk_mk": "Macedonian", "ml_in": "Malayalam", "mn_mn": "Mongolian", "mr_in": "Marathi", "ms_my": "Malay", "mt_mt": "Maltese", "my_mm": "Myanmar", "nb_no": "Norwegian", "ne_np": "Nepali", "nl_nl": "Dutch", "oc_fr": "Occitan", "pa_in": "Punjabi", "pl_pl": "Polish", "ps_af": "Pashto", "pt_br": "Portuguese", "ro_ro": "Romanian", "ru_ru": "Russian", "sd_in": "Sindhi", "sk_sk": "Slovak", "sl_si": "Slovenian", "sn_zw": "Shona", "so_so": "Somali", "sr_rs": "Serbian", "sv_se": "Swedish", "sw_ke": "Swahili", "ta_in": "Tamil", "te_in": "Telugu", "tg_tj": "Tajik", "th_th": "Thai", "tr_tr": "Turkish", "uk_ua": "Ukrainian", "ur_pk": "Urdu", "uz_uz": "Uzbek", "vi_vn": "Vietnamese", "yo_ng": "Yoruba"}
selection = widgets.Dropdown(
    options=[("Select language", None), ("----------", None)] + sorted([(f"{v} ({k})", k) for k, v in languages.items()]),
    value="hi_in",
    description='Language:',
    disabled=False,
)#"hi_in": "Hindi"

selection

Dropdown(description='Language:', index=29, options=(('Select language', None), ('----------', None), ('Afrika…

In [4]:
lang = selection.value
language = languages[lang]

assert lang is not None, "Please select a language"
print(f"Selected language: {language} ({lang})")

Selected language: Hindi (hi_in)


In [5]:
class Fleurs(torch.utils.data.Dataset):
    """
    A simple class to wrap Fleurs and subsample a portion of the dataset as needed.
    """
    def __init__(self, lang, split="test", subsample_rate=1, device=DEVICE):
        url = f"https://storage.googleapis.com/xtreme_translations/FLEURS102/{lang}.tar.gz"
        tar_path = os.path.expanduser(f"~/.cache/fleurs/{lang}.tgz")
        os.makedirs(os.path.dirname(tar_path), exist_ok=True)

        if not os.path.exists(tar_path):
            with urllib.request.urlopen(url) as source, open(tar_path, "wb") as output:
                with tqdm(total=int(source.info().get("Content-Length")), ncols=80, unit='iB', unit_scale=True, unit_divisor=1024) as loop:
                    while True:
                        buffer = source.read(8192)
                        if not buffer:
                            break

                        output.write(buffer)
                        loop.update(len(buffer))

        labels = {}
        all_audio = {}
        with tarfile.open(tar_path, "r:gz") as tar:
            for member in tar.getmembers():
                name = member.name
                if name.endswith(f"{split}.tsv"):
                    labels = pd.read_table(tar.extractfile(member), names=("id", "file_name", "raw_transcription", "transcription", "_", "num_samples", "gender"))

                if f"/{split}/" in name and name.endswith(".wav"):
                    audio_bytes = tar.extractfile(member).read()
                    all_audio[os.path.basename(name)] = wavfile.read(io.BytesIO(audio_bytes))[1]
                    

        self.labels = labels.to_dict("records")[::subsample_rate]
        self.all_audio = all_audio
        self.device = device

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, item):
        record = self.labels[item]
        audio = torch.from_numpy(self.all_audio[record["file_name"]].copy())
        text = record["transcription"]
        
        return (audio, text)

In [6]:
#Downloading the dataset
dataset = Fleurs(lang, subsample_rate=10) 

  0%|                                              | 0.00/1.53G [00:00<?, ?iB/s]

In [7]:
#Downloading the whisper multilingual midium varient model
model = whisper.load_model("medium")
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

100%|█████████████████████████████████████| 1.42G/1.42G [00:36<00:00, 41.4MiB/s]


Model is multilingual and has 762,321,920 parameters.


In [8]:
#creating ditionaries for transcibtion as well as traslation
options = dict(language=language, beam_size=5, best_of=5)
transcribe_options = dict(task="transcribe", **options)
translate_options = dict(task="translate", **options)

In [9]:
references = []
transcriptions = []
translations = []

for audio, text in tqdm(dataset):
    transcription = model.transcribe(audio, **transcribe_options)["text"]
    translation = model.transcribe(audio, **translate_options)["text"]
    
    transcriptions.append(transcription)
    translations.append(translation)
    references.append(text)

  0%|          | 0/42 [00:00<?, ?it/s]

In [10]:
#output dataframe
data = pd.DataFrame(dict(reference=references, transcription=transcriptions, translation=translations))
data

,reference,transcription,translation
0,स्कीइंग मार्ग को एक हाईकिंग लंबी पैदल यात्रा मार्ग जैसा ही सोचें।,अस्कीन्मार्क को एक हाईकिन् लंबी पैदल यात्रा मार्ग जैसा ही सोचें।,Think of skiing as a long hiking trail.
1,अधिकांश छोटे द्वीप स्वतंत्र राष्ट्र हैं या फ़्रांस से संबंधित हैं और लग्ज़री बीच रिसॉर्ट के रूप में जाने जाते हैं,अधिकाश चोटे दे पस्वतंत्र राष्टर हैं या फ्रांस से सम्बंदित हैं और लक्स्री बीट्स रिसॉर्ट के रूप में जाने जाते हैं.,Most of them are related to the small and independent state or France and are known as Luxury Beach Resort.
2,तूफान और बवंडर की तरह आंधी ओले भारी बारिश और जंगल की आग तीव्र मौसम का हिस्सा और असर हैं,"तूफान और भवंदर की तरा आंधी, आूले, भारी, बारिश और जंगल की आक तीवर मौसम का हिस्सा और असर है।","Like storms and storms, winds, storms, heavy rains and forest fires are part and parcel of severe weather."
3,महिलाएं यह अनुशंसा की जाती है कि कोई भी महिला यात्री वास्तविक वैवाहिक स्थिति के बावजूद कहती है कि वह विवाहित है,महिलाये या अनुश्रणशा की जाती है कि कोई भी महिलायाइादरी वास्तविक विवाई क्षिति के बावजुत कहती है कि वो विवाईत है।,Women are said to be married despite the fact that they are married.
4,वाइल्डलाइफ़ हैबिटेंट्स के रूप में काम करने वाली रेती और तटों को बनाने के लिए गाद ज़रूरी थी,Wildlife habitants के रूप में काम करने वाली रेती और तटों को बनाने के लिए गाद जुरूडी थी।,It was necessary to build a guard to work as wildlife habitants.
5,बाली में इस एजेंडे के अन्य विषयों में दुनिया के बचे हुए जंगलों को बचाने और ऐसी तकनीकों का आदान प्रदान करना का विषय शामिल है जिससे कि विकासशील देशों को कम प्रदूषणकारी तरीकों से आगे बढ़ने में मदद मिले,"बाली में इस अजेंडे के अन्य विष्यों में दुन्या के बचे हुए जंगलों को बचाने और ऐसी तेक्नीकों का अधान पर्दान करने का विष्य शामिल हैं, जिससे कि विकास जीर देशों को कम परदुषन कारी तरीकों से आगे बरने में मदद मेले.","In this agenda, Bali is involved in protecting the remaining forests of the world and providing solutions to such techniques so that developing countries can move forward in less polluting ways."
6,1889 में यह बंदरगाह कुख्यात नौसैनिक गतिरोध का ठिकाना था उस समय जर्मनी अमेरिका और ब्रिटेन के सात जहाजों ने इस बंदरगाह से जाने से इनकार कर दिया था,1889 में यह बंदर्गाख युक्यात नोसेनिक गतिरोत का थिकाना था। इस बंदर्गाख से इनकार कर दिया था।,"In 1889, this was the place of the 9th World War, during which Germany, America and Britain refused to leave this place."
7,सन 1976 तक माचू पिचू के तीस प्रतिशत हिस्से का जीर्णोद्धार कर दिया गया था और जीर्णोद्धार का कार्य आज तक जारी है,सं 1976 तक माचु पिचु के 30% हिस्से का जिर्नोदार कर दिया घया था और जिर्नोदार का कारे आस तक जारी है।,"In 1976, 30% of Machu Picchu's share of Jirnodhar was done and the work of Jirnodhar is still going on."
8,ms बीमारी केंद्रीय तंत्रिका तंत्र पर असर करती है जिसमें दिमाग स्पाइनल कॉर्ड और ऑप्टिक नर्व शामिल हैं,"MS बिमारी केंद्रिय तंत्र का तंत्र पर असर करती है, जिसमें दिमाक, स्पाइनल कौड और अप्टिक नर्व शामिल है।","The MS disease center affects the immune system, which includes the brain, spinal cord and optic nerve."
9,समाजीकरण के महत्व को स्पष्ट करने के लिए इस्तेमाल किए जाने वाले सबसे आम तरीकों में से कुछ बच्चों के दुर्भाग्यपूर्ण मामलों को आकर्षित करना है जो बड़े होने के दौरान वयस्कों द्वारा उपेक्षित नहीं बल्कि उपेक्षा दुर्भाग्य या दुर्व्यवहार के माध्यम से होते थे,"समाजी करन के महत्तो को इस्पष्ट करने के लिए इस्तमाल किया जाने वाले सबसे आम तरीकों में से कुछ बच्चों के दृर्भाग्य पुर्न मामलों को आकर्षित करना हैं, जो बड़े होने के दौरान वैस्को दौरा उपेक्छित नहीं बल्कि उपेक्छा दृर्भाग्य या दृर्वेवार के माध्यम से होते थे।","The most common methods used to clarify the importance of socialization are to attract the unfortunate events of some children, which did not occur due to age, but due to unfortunate or misbehaviour."


In [ ]:
#P.S. Above Example referenced from the official whisper github